In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import torch
from torchmultimodal.models.flava.model import flava_model_for_classification, flava_model_for_pretraining

In [ ]:
from torchvision.transforms import ToPILImage
from transformers import BertTokenizer

tokernizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
import os
import sys
import matplotlib.pyplot as plt

In [ ]:
sys.path.append("../..")

In [ ]:
a = torch.load("flava-epoch=00-step=21000.ckpt", map_location=torch.device('cpu'))

In [ ]:
from flava.model import FLAVAPreTrainingLightningModule
from flava.data.datamodules import ISICDataset
from flava.data.transforms import default_image_pretraining_transforms

In [ ]:
m = FLAVAPreTrainingLightningModule()

In [ ]:
m.load_state_dict(a['state_dict'])

In [ ]:
ds = ISICDataset("/home/ravi/Documents/courses/healthcare/Research/test_data.csv",
                 "/home/ravi/Documents/courses/healthcare/Research/archive/train",
                 "image",
                  True)
tfms = default_image_pretraining_transforms()[1]
ds.set_transform(tfms)
dl = torch.utils.data.DataLoader(
            ds,
            batch_size=1,
            num_workers=1,
            sampler=None,
            shuffle=False,
            # collate_fn=_build_collator(),
            # uneven batches can cause distributed issues,
            # drop last batch to prevent those.
            drop_last=False,
        )
to_pil = ToPILImage()

In [ ]:
im_ = ds[0]['image']

In [ ]:
for data in dl:
    r = m._step(data, 1)
    break

In [ ]:
m.model.model.image_encoder(data['image']).last_hidden_state[:, 0].squeeze()

In [ ]:
data.keys()

In [ ]:
data['label'], data['img_pth']

In [ ]:
plt.imshow(to_pil(im_).convert('L'), cmap="gray")

In [ ]:
plt.imshow(plt.imread(ds[0]['img_pth']))